### 목표
- 서울시 각 구별 CCTV수를 파악해보자.(많은/적은)
- 인구대비 CCTV가 많은/적은 지역을 파악해보자
- CCTV 예측 값을 확인하고, CCTV가 예측 값 대비 많은/적은 파악해보자.

In [128]:
import numpy as np     # 데이터를 순차적으로 저장, 빠른 연산속도, 다양한 기능
import pandas as pd    # 데이터를 표 형태로 다룰때 사용
import matplotlib.pyplot as plt      # 데이터 시각화(python에서)
from matplotlib import rc          # for 한글 폰트 지정
rc('font', family="Malgun Gothic")    #font:글자체(맑은고딕 기본) 
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', None)       # 특별히 수치를 안하겠다고 쓰는것 즉. 데이터의 크기에 맞춰서 알아서 지정한다는거

# 0. 데이터 로딩

### 서울시 구별 CCTV 데이터

In [129]:
CCTV_Seoul = pd.read_csv("CCTV_in_Seoul.csv", encoding="utf-8")         #csv:'comma-seperated variables'
CCTV_Seoul.tail()            #범위 지정안했을때 끝값5개 출력

,기관명,소계,2013년도 이전,2014년,2015년,2016년
20,용산구,1624,1368,218,112,398
21,은평구,1873,1138,224,278,468
22,종로구,1002,464,314,211,630
23,중구,671,413,190,72,348
24,중랑구,660,509,121,177,109


### 서울시 인구현황 데이터

In [130]:
pop_Seoul = pd.read_excel('population_in_Seoul.xls')   #첫줄만 column이라 생각하고 2,3행도 data로 간주하여 불러들임.
pop_Seoul.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2.36,1321458
3,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2.11,25425
4,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2.09,20764


#### 옵션 설정
- header : 읽고 싶은 row index (0부터 시작)
- usecols : 읽고 싶은 column 선택

In [131]:
pop_Seoul = pd.read_excel('population_in_Seoul.xls',
                         header = 2,                           #2행부터 가져와라
                         usecols = "B, D, G, J, N")            #이 다섯 column만 가져와라
pop_Seoul.head()

,자치구,계,계.1,계.2,65세이상고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


In [132]:
pop_Seoul.columns = ['구 이름','전체 인구수','한국인수','외국인수','고령자수']

In [133]:
pop_Seoul.head()

,구 이름,전체 인구수,한국인수,외국인수,고령자수
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


In [134]:
CCTV_col = list(CCTV_Seoul.columns)

In [135]:
CCTV_col[0] = "구 이름"

In [136]:
CCTV_col

['구 이름', '소계', '2013년도 이전', '2014년', '2015년', '2016년']

In [137]:
CCTV_Seoul.columns = CCTV_col

In [138]:
CCTV_Seoul.head()

,구 이름,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


##### cf. rename

In [139]:
CCTV_Seoul = CCTV_Seoul.rename(columns={"소계":"CCTV총계"})

In [140]:
CCTV_Seoul.head()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


# 1. CCTV 수가 많은/적은 파악해보자. (각각 5개)

#### 많은 지역 5개

In [141]:
CCTV_Seoul.sort_values(by="CCTV총계", ascending=False).head()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
18,양천구,2034,1843,142,30,467
14,서초구,1930,1406,157,336,398
21,은평구,1873,1138,224,278,468
20,용산구,1624,1368,218,112,398


#### 적은 지역 4개

In [142]:
CCTV_Seoul.sort_values(by="CCTV총계", ascending=False).tail()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년
23,중구,671,413,190,72,348
24,중랑구,660,509,121,177,109
17,송파구,618,529,21,68,463
12,마포구,574,314,118,169,379
9,도봉구,485,238,159,42,386


# 2. 인구대비 CCTV 많은/적은 지역 파악

In [143]:
pop_Seoul = pop_Seoul.drop(0)        #합계 삭제하기

In [144]:
pop_Seoul['구 이름'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', nan],
      dtype=object)

## 결측치 확인하기

In [145]:
pop_Seoul[pop_Seoul['구 이름'].isnull()]

,구 이름,전체 인구수,한국인수,외국인수,고령자수
26,NaN,NaN,NaN,NaN,NaN


In [146]:
pop_Seoul = pop_Seoul.drop(26)

### CCTV 구 이름과 인구수 구 이름이 똑같은지 확인

###### 집합 연산을 이용하자.
- 차집합 : -
- 교집합 : &
- 합집합 : |

In [147]:
CCTV_구_이름_집합 = set(CCTV_Seoul['구 이름'])
pop_구_이름_집합 = set(pop_Seoul ['구 이름'])

In [148]:
CCTV_구_이름_집합 - pop_구_이름_집합

set()

In [149]:
pop_구_이름_집합 - CCTV_구_이름_집합

set()

In [150]:
data_result = pd.merge(CCTV_Seoul,pop_Seoul, on="구 이름",how='inner')      
#단순히 붙이는작업:concat
#특정 열을 기준으로 병합하려 할때 merge

In [151]:
data_result.head()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년,전체 인구수,한국인수,외국인수,고령자수
0,강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0
1,강동구,773,379,99,155,377,453233.0,449019.0,4214.0,54622.0
2,강북구,748,369,120,138,204,330192.0,326686.0,3506.0,54813.0
3,강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0
4,관악구,1496,846,260,390,613,525515.0,507203.0,18312.0,68082.0


### 인구수 대비 CCTV 비율을 계산

In [152]:
# 특성공학 (feature engineering): 어떤column들끼리 연산하여 새롭게 무언가 만들어 내는 것
data_result['인구당CCTV비율'] = data_result['CCTV총계']/data_result["전체 인구수"]

In [153]:
data_result.head()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년,전체 인구수,한국인수,외국인수,고령자수,인구당CCTV비율
0,강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0,0.004873
1,강동구,773,379,99,155,377,453233.0,449019.0,4214.0,54622.0,0.001706
2,강북구,748,369,120,138,204,330192.0,326686.0,3506.0,54813.0,0.002265
3,강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0,0.001464
4,관악구,1496,846,260,390,613,525515.0,507203.0,18312.0,68082.0,0.002847


### 인구수 대비 CCTV가 많은 지역

In [154]:
data_result.sort_values(by='인구당CCTV비율', ascending=False).head()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년,전체 인구수,한국인수,외국인수,고령자수,인구당CCTV비율
20,용산구,1624,1368,218,112,398,244203.0,229456.0,14747.0,36231.0,0.006650
22,종로구,1002,464,314,211,630,162820.0,153589.0,9231.0,25425.0,0.006154
23,중구,671,413,190,72,348,133240.0,124312.0,8928.0,20764.0,0.005036
0,강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0,0.004873
14,서초구,1930,1406,157,336,398,450310.0,445994.0,4316.0,51733.0,0.004286


### 인구수 대비 CCTV가 적은 지역

In [155]:
data_result.sort_values(by='인구당CCTV비율', ascending=False).tail()

,구 이름,CCTV총계,2013년도 이전,2014년,2015년,2016년,전체 인구수,한국인수,외국인수,고령자수,인구당CCTV비율
24,중랑구,660,509,121,177,109,414503.0,409882.0,4621.0,56774.0,0.001592
12,마포구,574,314,118,169,379,389649.0,378566.0,11083.0,48765.0,0.001473
3,강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0,0.001464
9,도봉구,485,238,159,42,386,348646.0,346629.0,2017.0,51312.0,0.001391
17,송파구,618,529,21,68,463,667483.0,660584.0,6899.0,72506.0,0.000926


# 3. CCTV 예측 값 만들기

### 상관관계 : 두 개의 특성(column) 사이의 '관계'를 분석

In [156]:
data_result.corr()        #계수 계산해줌. 1-1부터 오른아래대각선으로는 1값=why?본인값이니깐. =>이 값 제외하고 보도록. 
#행 or 열로 볼 것.

,CCTV총계,2013년도 이전,2014년,2015년,2016년,전체 인구수,한국인수,외국인수,고령자수,인구당CCTV비율
CCTV총계,1.000000,0.862756,0.450062,0.624402,0.593398,0.306342,0.304287,-0.023786,0.255196,0.575784
2013년도 이전,0.862756,1.000000,0.121888,0.257748,0.355482,0.168177,0.163142,0.048973,0.105379,0.577185
2014년,0.450062,0.121888,1.000000,0.312842,0.415387,0.027040,0.025005,0.027325,0.010233,0.399458
2015년,0.624402,0.257748,0.312842,1.000000,0.513767,0.368912,0.363796,0.013301,0.372789,0.207860
2016년,0.593398,0.355482,0.415387,0.513767,1.000000,0.144959,0.145966,-0.042688,0.065784,0.438115
전체 인구수,0.306342,0.168177,0.027040,0.368912,0.144959,1.000000,0.998061,-0.153371,0.932667,-0.514143
한국인수,0.304287,0.163142,0.025005,0.363796,0.145966,0.998061,1.000000,-0.214576,0.931636,-0.513169
외국인수,-0.023786,0.048973,0.027325,0.013301,-0.042688,-0.153371,-0.214576,1.000000,-0.155381,0.079211
고령자수,0.255196,0.105379,0.010233,0.372789,0.065784,0.932667,0.931636,-0.155381,1.000000,-0.541024
인구당CCTV비율,0.575784,0.577185,0.399458,0.207860,0.438115,-0.514143,-0.513169,0.079211,-0.541024,1.000000


### 인구수를 기반으로 CCTV 수를 예측
- 약한 양의 상관관계
- 추후에 더 많은 column을 수집해 추가적으로 분석가능

#### 선형회귀

In [157]:
# 인덱스를 나중에 설정
data_result = data_result.set_index('구 이름')

In [158]:
data_result.head()

,CCTV총계,2013년도 이전,2014년,2015년,2016년,전체 인구수,한국인수,외국인수,고령자수,인구당CCTV비율
구 이름,,,,,,,,,,
강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0,0.004873
강동구,773,379,99,155,377,453233.0,449019.0,4214.0,54622.0,0.001706
강북구,748,369,120,138,204,330192.0,326686.0,3506.0,54813.0,0.002265
강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0,0.001464
관악구,1496,846,260,390,613,525515.0,507203.0,18312.0,68082.0,0.002847


In [159]:
from sklearn.linear_model import LinearRegression

In [160]:
linear_model = LinearRegression()    # 예측하는 모델 생성

In [161]:
linear_model.fit(data_result[['전체 인구수']], data_result['CCTV총계'])   # 적절한 데이터 넣으면 알아서 공식 만들어줌

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [162]:
data_result[['전체 인구수']]

,전체 인구수
구 이름,
강남구,570500.0
강동구,453233.0
강북구,330192.0
강서구,603772.0
관악구,525515.0
광진구,372164.0
구로구,447874.0
금천구,255082.0
노원구,569384.0


In [163]:
pre_CCTV = linear_model.predict(data_result[['전체 인구수']])

In [164]:
error_CCTV = pre_CCTV - data_result['CCTV총계']    #예측값-실제값 =>음수값:예측보다<실제설치,   양수값: 예측보다> 실제 더 적게 설치

In [165]:
error_CCTV.sort_values()                           #예측값-실제값 =>음수값:예측보다<실제설치,   양수값: 예측보다> 실제 더 적게 설치

구 이름
강남구    -1388.055355
양천구     -760.563512
서초구     -695.403794
용산구     -659.231690
은평구     -580.698457
구로구     -329.592918
성북구     -215.068447
동대문구    -165.202586
관악구     -162.948104
종로구     -143.775396
금천구      -35.989293
성동구       -9.464016
동작구       94.122892
서대문구     111.376568
노원구      125.483618
중구       148.499528
영등포구     268.640012
강북구      329.342026
광진구      425.290264
강동구      465.422892
중랑구      527.718965
강서구      551.503155
마포구      581.180999
도봉구      616.501341
송파구      900.911312
Name: CCTV총계, dtype: float64